In [91]:
%load_ext autoreload
%autoreload 2

In [88]:
import os

import pyarabic.araby as araby

from langchain_community.document_loaders import PDFPlumberLoader
from langchain_core.documents import Document

In [7]:
raw_docs_path = "../../data/raw"

In [8]:
os.listdir(raw_docs_path)

['Actual Budget Report 2022.pdf',
 'Press Release - 2022 Results (Stock Market).pdf']

In [11]:
docs = [PDFPlumberLoader(os.path.join(raw_docs_path, p)).load() for p in os.listdir(raw_docs_path)]

In [12]:
docs[0]

[Document(page_content='رـــــــــــــــيرقت\nيلعفلا ةينازيملا ءادأ\nم2022 يــــلاملا ماــــــعلا ةــــــياهنل\n', metadata={'source': '../../data/raw\\Actual Budget Report 2022.pdf', 'file_path': '../../data/raw\\Actual Budget Report 2022.pdf', 'page': 0, 'total_pages': 18, 'CreationDate': "D:20230612151620+03'00'", 'Creator': 'Adobe InDesign 18.3 (Windows)', 'ModDate': "D:20230612151624+03'00'", 'Producer': 'Adobe PDF Library 17.0', 'Trapped': 'False'}),
 Document(page_content='03 ةمدقم\n04 يذيفنتلا صخلملا\n06 ةيداصتقلاا تارشؤملا :لاً وأ\n10 ةماعلا ةيلاملا ءادأ :اًيناث\n10 تاداريلإا .أ\n12 تاقفنلا .ب\n16 ليومتلاو ةينازيملا ضئاف .ج\n', metadata={'source': '../../data/raw\\Actual Budget Report 2022.pdf', 'file_path': '../../data/raw\\Actual Budget Report 2022.pdf', 'page': 1, 'total_pages': 18, 'CreationDate': "D:20230612151620+03'00'", 'Creator': 'Adobe InDesign 18.3 (Windows)', 'ModDate': "D:20230612151624+03'00'", 'Producer': 'Adobe PDF Library 17.0', 'Trapped': 'False'}),
 Document

In [13]:
len(docs[0])

18

The first document contains 18 pages, and the loaded document contains 18, which means that every page was parsed individually.

In [16]:
print(docs[0][2].page_content)

ةــمدقم
ضارعتــسلا م2022 ماعل يلعفلا ةينازيملا ءادلأ ماعلا ةــياهن رــيرقت رادــصإب ةــيلاملا ةرازو موــقت
يــلاملا ماــعلا لاــخ داــصتقلااو ةــماعلا ةــيلاملا يــبناج ىــلع يــلعفلا ءادلأا تارــشؤمو تاــنايب
ماــعلل ةــينازيملل ةدــمتعملا تارــيدقتلا نــع فاــتخلاا بابــسأ زرــبلأ ضرــع ىــلإ ةــفاضلإاب ،م2022
.م2022 يــلاملا
ةــماعلا ةــينازيملا نــع ةــيلاملا ةرازو اــهردصت يــتلا رــيراقتلا ةلــسلس نــمض رــيرقتلا اذــه يــتأيو
اــمبو ةــموكحلا اــهجهتنت يــتلا يــلاملا حاــصفلإاو ةيفافــشلا تارداــبمل اًــمعدو ،ةــكلمملا يــف
اذــه يــف تارداــبم ةدــعب ةــيلاملا ةرازو تــماق ،2030 ةــكلمملا ةــيؤر تافدهتــسم عــم قــستي
ءادلأا نــع ةــيرود رــيراقت رــشنو رادــصإ لاــخ نــم ةــيضاملا ةرــتفلا لاــخ رــبكأ لكــشبو ،لاــجملا
يــتلا تارداــبملاو تاــسايسلا نــع حاــصفلإا ىوتــسم رــيوطتو ،ةــلودلل ةــماعلا ةــينازيملل يــلاملا
ناــيبو ،يونــسلا ةــينازيملا ناــيب لــثم اــهرادصإب موــقت يــتلا رــيراقتلا رــيوطتو ،اــهذيفنتب موــقت
رــيرقتو ،ةيونــس عــبرلا ءادلأا رــيراقت ىــلإ ةـ

Pdfplumber parses that text line by line from left to right, which means that we should flip each line, while preserving numbers and any english text if present.

In [40]:
lines = docs[0][3].page_content.split('\n')
lines[1]

'عافتراو ،وـــمنلا تلادعم يف ؤطابت اـــهزربأ تايدحت نـــم يملاعلا داصتقلاا هـــهجاوي ام ءوـــض يـــف'

In [20]:
lines[1][::-1]

'تقــوم وزارة الماليــة بإصــدار تقريــر نهايــة العام ألداء الميزانية الفعلي لعام 2202م الســتعراض'

In [104]:
import re

def flip_arabic_line(line):

    def reverse_number(match):
        # Reverse the number found
        num = match.group(0)
        return num[::-1]

    line = line[::-1]
    
    # New line
    new_line = []

    for word in re.split(r'(\s+)', line):

        if re.match(r'[+-]?[\d,]+(?:\.\d+)?', word):
            
            # Remove commas as it makes matching numbers and reversing them easier
            # This could be removed and the function that reverse numbers could 
            # be modified to account for commas
            number = word.replace(',', '')

            # Use regex to find all numbers and apply the reversing function
            number = re.sub(r'[+-]?[0-9]+(?:\.[0-9]+)?', reverse_number, number)
                
            new_line.append(number)

        elif re.search(r'[A-Za-z]+', word):
            new_line.append(word[::-1])

        else:
            new_line.append(word)
    
    # Join the reversed words to form the sentence
    new_line = ''.join(new_line)
    
    return new_line

In [105]:
lines[15]

'نـــسّ حتو يفاعت ىلإ كلذ دوعيو ،ةدمتعملا ةينازيملاب ةنراقم %14.2 ةبـــسنب ةيطفنلا ريغ تاداريلإا'

In [106]:
flipped_line = flip_arabic_line(lines[17])
flipped_line

'اإليـــرادات غيـــر النفطيـــة، باإلضافة إلـــى التطور فـــي اإلدارة الضريبية.'

In [63]:
def flip_arabic_text(text):
    return '\n'.join(flip_arabic_line(l) for l in text.split('\n'))

In [64]:
flipped_page = flip_arabic_text(docs[0][3].page_content)

In [65]:
print(flipped_page)

الملخص التنفيذي
فـــي ضـــوء ما يواجهـــه االقتصاد العالمي مـــن تحديات أبرزهـــا تباطؤ في معدالت النمـــو، وارتفاع
معـــدالت التضخـــم وما صاحبه من قيام عدة بنوك مركزية بتشـــديد السياســـة النقديـــة، وحالة عدم
اليقيـــن فـــي ظـــل التوتـــرات الجيوسياســـية القائمـــة، إ ّال أن انعكاســـات هـــذه التحديات ُيعـــد محدودًا
علـــى االقتصـــاد المحلـــي، ويعود ذلـــك لإلصالحـــات الهيكلية التي قامـــت حكومة المملكـــة العربية
الســـعودية بتنفيذهـــا منـــذ انطـــالق الرؤيـــة، وكذلـــك تطبيق عدد من السياســـات بشـــكل اســـتباقي
والتطـــورات الملموســـة التـــي شـــهدتها المملكة خـــالل العـــام 2022م على الصعيديـــن االقتصادي
والمالـــي، حيـــث شـــهد االقتصـــاد الســـعودي أداء فـــاق التوقعـــات المحليـــة والدولية للعـــام المالي
ً
2022م. كمـــا انعكـــس األداء االقتصـــادي اإليجابـــي واإلصالحـــات الماليـــة في تقويـــة المركز المالي
للمملكـــة وإتاحـــة مســـاحة ماليـــة إضافيـــة للتعامل مـــع الصدمات في المســـتقبل من خـــالل تحقيق
فوائـــض فـــي الميزانيـــة ألول مرة منذ 8 سـ

It looks like some characters are reversed in "اإليـــرادات".

Also there are tatweels and other arabic text characterstics that should be normalized.

In [61]:
araby.strip_tatweel(flipped_line)

'اإليرادات غير النفطية، باإلضافة إلى التطور في اإلدارة الضريبية.'

In [89]:
def clean_arabic_text(text, **kwargs):
    if kwargs.get('diacritics', True): text = araby.strip_diacritics(text)
    if kwargs.get('harakat', True): text = araby.strip_harakat(text)
    if kwargs.get('lastharaka', True): text = araby.strip_lastharaka(text)
    if kwargs.get('shadda', True): text = araby.strip_shadda(text)
    if kwargs.get('small', True): text = araby.strip_small(text)
    if kwargs.get('tashkeel', True): text = araby.strip_tashkeel(text)
    if kwargs.get('tatweel', True): text = araby.strip_tatweel(text)

    text = re.sub(r'اال', 'الا', text)
    text = re.sub(r'اإل', 'الإ', text)
    text = re.sub(r'األ', 'الأ', text)
    text = re.sub(r'الل', 'لال', text)
    text = re.sub('ألول', 'لأول', text)
    return text

In [84]:
cleaned_page = clean_arabic_text(flipped_page)

In [85]:
print(cleaned_page)

الملخص التنفيذي
في ضوء ما يواجهه الاقتصاد العالمي من تحديات أبرزها تباطؤ في معدالت النمو، وارتفاع
معدالت التضخم وما صاحبه من قيام عدة بنوك مركزية بتشديد السياسة النقدية، وحالة عدم
اليقين في ظل التوترات الجيوسياسية القائمة، إ ال أن انعكاسات هذه التحديات يعد محدودا
على الاقتصاد المحلي، ويعود ذلك لإلصالحات الهيكلية التي قامت حكومة المملكة العربية
السعودية بتنفيذها منذ انطالق الرؤية، وكذلك تطبيق عدد من السياسات بشكل استباقي
والتطورات الملموسة التي شهدتها المملكة خلال العام 2022م على الصعيدين الاقتصادي
والمالي، حيث شهد الاقتصاد السعودي أداء فاق التوقعات المحلية والدولية للعام المالي

2022م. كما انعكس الأداء الاقتصادي الإيجابي والإصالحات المالية في تقوية المركز المالي
للمملكة وإتاحة مساحة مالية إضافية للتعامل مع الصدمات في المستقبل من خلال تحقيق
فوائض في الميزانية لأول مرة منذ 8 سنوات.
شهد أداء المالية العامة في عام 2022م ارتفاعا في إجمالي الإيرادات بحوالي 21.3%
عن الميزانية المعتمدة؛ نتيجة زيادة كل من الإيرادات النفطية وغير النفطية، ويعزى الارتفاع
في الإيرادات النفطية بشكل رئيس إلى ارتفاع أ

Now let's check the entire document.

In [86]:
for i, page in enumerate(docs[0]):
    print('-'*100)
    print('PAGE', i+1)
    print(clean_arabic_text(flip_arabic_text(page.page_content)))

----------------------------------------------------------------------------------------------------
PAGE 1
تقرير
أداء الميزانية الفعلي
لنهاية العام المالي 2022م

----------------------------------------------------------------------------------------------------
PAGE 2
مقدمة 03
الملخص التنفيذي 04
أو ال: المؤشرات الاقتصادية 06
ثانيا: أداء المالية العامة 10
أ. الإيرادات 10
ب. النفقات 12
ج. فائض الميزانية والتمويل 16

----------------------------------------------------------------------------------------------------
PAGE 3
مقدمة
تقوم وزارة المالية بإصدار تقرير نهاية العام ألداء الميزانية الفعلي لعام 2022م الستعراض
بيانات ومؤشرات الأداء الفعلي على جانبي المالية العامة والاقتصاد خال العام المالي
2022م، بالإضافة إلى عرض ألبرز أسباب الاختاف عن التقديرات المعتمدة للميزانية للعام
المالي 2022م.
ويأتي هذا التقرير ضمن سلسلة التقارير التي تصدرها وزارة المالية عن الميزانية العامة
في المملكة، ودعما لمبادرات الشفافية والإفصاح المالي التي تنتهجها الحكومة وبما
يتسق مع مستهدفات رؤية المملكة 2030، قامت 

In [87]:
for i, page in enumerate(docs[1]):
    print('-'*100)
    print('PAGE', i+1)
    print(clean_arabic_text(flip_arabic_text(page.page_content)))

----------------------------------------------------------------------------------------------------
PAGE 1
ﺑﻴﺎن أرﺑﺎح اﻟﻨﺘﺎﺋﺞ اﻟﻤﺎﻟﻴﺔ اﻟﺴﻨﻮﻳﺔ
ﻟﻠﻌﺎم 2022م
26 ﻓﺒﺮاﻳﺮ 2023م

----------------------------------------------------------------------------------------------------
PAGE 2
ﺑﻴﺎن أرﺑﺎح اﻟﻨﺘﺎﺋﺞ اﻟﻤﺎﻟﻴﺔ اﻟﺴﻨﻮﻳﺔ ﻟﻠﻌﺎم 2022م
26 ﻓﺒﺮاﻳﺮ 2023م
ﻣﺠﻤﻮﻋﺔ ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ ﺗﻌﻠﻦ ﻋﻦ ﻧﺘﺎﺋﺠﻬﺎ اﻟﻤﺎﻟﻴﺔ اﻟﺴﻨﻮﻳﺔ ﻟﻠﻌﺎم 2022م وﺗﺤﻘﻖ ﺻﺎﻓﻲ رﺑﺢ
ﺑﻌﺪ اﻟﺰﻛﺎة ﺑﻘﻴﻤﺔ 424.6 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي
• أﻋﻠﻨﺖ ﻣﺠﻤﻮﻋﺔ ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ ﻋﻦ ﻣﺸﺎرﻛﺘﻬﺎ ﻣﻊ ﺻﻨﺪوق اﻻﺳﺘﺜﻤﺎرات اﻟﻌﺎﻣﺔ ﻓﻲ ﺗﺄﺳﻴﺲ ﺷﺮﻛﺔ ﺳﻮق
اﻟﻜﺮﺑﻮن اﻟﻄﻮﻋﻲ اﻻﻗﻠﻴﻤﻴﺔ .
• اﻛﺘﻤﺎل أول إدراج ﻣﺰدوج وﻣﺘﺰاﻣﻦ ﺑﻴﻦ ﺷﺮﻛﺔ ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ وﺳﻮق أﺑﻮﻇﺒﻲ ﻟﻸوراق اﻟﻤﺎﻟﻴﺔ .
• وﻗﻌﺖ ﺷﺮﻛﺔ "واﻣﺾ"، إﺣﺪى اﻟﺸﺮﻛﺎت اﻟﺘﺎﺑﻌﺔ ﻟﻤﺠﻤﻮﻋﺔ ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ، اﺗﻔﺎﻗﻴﺔ ﺑﻴﻊ وﺷﺮاء ﻣﻊ ﺷﺮﻛﺔ
"اﻟﻤﺒﺎدرات اﻟﺜﺎﻧﻴﺔ ﻟﻼﺳﺘﺜﻤﺎر" ﻟﻼﺳﺘﺤﻮاذ ﻋﻠﻰ 51% ﻣﻦ رأس اﻟﻤﺎل اﻟﻤﺼﺪر ﻟﺸﺮﻛﺔ ﺷﺒﻜﺔ ﻣﺒﺎﺷﺮ اﻟﻤﺎﻟﻴﺔ "داﻳﺮﻛﺖ
إف إن" .
• إﻃﻼق إﻃﺎر ﻋﻤﻞ ﺻﺎﻧﻊ اﻟﺴﻮق ﻷﺳﻮاق اﻷﺳﻬﻢ واﻟﻤﺸﺘﻘﺎت اﻟﻤﺎﻟﻴﺔ، ﻛﺠﺰء ﻣﻦ ﺟﻬﻮد ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ ﻟﻀﻤﺎن
ﺗﻮﻓﺮ اﻟﺴﻴﻮﻟﺔ وزﻳﺎدة ﻛﻔﺎءة ﺗﺤﺪﻳﺪ اﻷﺳﻌﺎر ﺗﻤﺎﺷﻴﺎ 

Since the end line of any document is a footer, it should be removed as it could result in false matches during retrieval.

Also since tables are always complete in one page, a specific page separator should be specified in order to enable correct splitting of documents that include entire tables.

In [90]:
def load_pdf(path, remove_footer=True, page_separator='\n\n\n', **kwargs):
    """
    Loads pdf pages using pdfplumber, preprocesses them and then concats them
    into one text with a specific separator.
    """
    pages = PDFPlumberLoader(path).load()

    merged_pages = []
    for p in pages:
        # Flip arabic text
        page_content = flip_arabic_text(page.page_content)

        # Clean arabic text
        page_content = clean_arabic_text(page_content, **kwargs)

        # Remove footer line
        page_content = '\n'.join(l for l in p.page_content.split('\n')[:-1])
        merged_pages.append(page_content)

    merged_pages = page_separator.join(merged_pages)
    
    # Create a langhchain document
    document = Document(page_content=merged_pages,
                        metadata={"source": path,
                                  "file_path": path})
    
    return document

In [95]:
import sys

sys.path.append('../../')

In [112]:
from src.ingestion import load_pdf

In [113]:
docs = [load_pdf(os.path.join(raw_docs_path, p)) for p in os.listdir(raw_docs_path)]

In [114]:
print(docs[0].page_content)

تقرير
أداء الميزانية الفعلي


مقدمة 03
الملخص التنفيذي 04
أو ال: المؤشرات الاقتصادية 06
ثانيا: أداء المالية العامة 10
أ. الإيرادات 10
ب. النفقات 12


مقدمة
تقوم وزارة المالية بإصدار تقرير نهاية العام ألداء الميزانية الفعلي لعام 2022م الستعراض
بيانات ومؤشرات الأداء الفعلي على جانبي المالية العامة والاقتصاد خال العام المالي
2022م، بالإضافة إلى عرض ألبرز أسباب الاختاف عن التقديرات المعتمدة للميزانية للعام
المالي 2022م.
ويأتي هذا التقرير ضمن سلسلة التقارير التي تصدرها وزارة المالية عن الميزانية العامة
في المملكة، ودعما لمبادرات الشفافية والإفصاح المالي التي تنتهجها الحكومة وبما
يتسق مع مستهدفات رؤية المملكة 2030، قامت وزارة المالية بعدة مبادرات في هذا
المجال، وبشكل أكبر خال الفترة الماضية من خال إصدار ونشر تقارير دورية عن الأداء
المالي للميزانية العامة للدولة، وتطوير مستوى الإفصاح عن السياسات والمبادرات التي
تقوم بتنفيذها، وتطوير التقارير التي تقوم بإصدارها مثل بيان الميزانية السنوي، وبيان
الميزانية التمهيدي، وكذلك نسخة المواطن، بالإضافة إلى تقارير الأداء الربع سنوية، وتقرير
الأداء المالي 

In [115]:
print(docs[1].page_content)

ﺑﻴﺎن أرﺑﺎح اﻟﻨﺘﺎﺋﺞ اﻟﻤﺎﻟﻴﺔ اﻟﺴﻨﻮﻳﺔ
ﻟﻠﻌﺎم 2022م


ﺑﻴﺎن أرﺑﺎح اﻟﻨﺘﺎﺋﺞ اﻟﻤﺎﻟﻴﺔ اﻟﺴﻨﻮﻳﺔ ﻟﻠﻌﺎم 2022م
26 ﻓﺒﺮاﻳﺮ 2023م
ﻣﺠﻤﻮﻋﺔ ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ ﺗﻌﻠﻦ ﻋﻦ ﻧﺘﺎﺋﺠﻬﺎ اﻟﻤﺎﻟﻴﺔ اﻟﺴﻨﻮﻳﺔ ﻟﻠﻌﺎم 2022م وﺗﺤﻘﻖ ﺻﺎﻓﻲ رﺑﺢ
ﺑﻌﺪ اﻟﺰﻛﺎة ﺑﻘﻴﻤﺔ 424.6 ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي
• أﻋﻠﻨﺖ ﻣﺠﻤﻮﻋﺔ ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ ﻋﻦ ﻣﺸﺎرﻛﺘﻬﺎ ﻣﻊ ﺻﻨﺪوق اﻻﺳﺘﺜﻤﺎرات اﻟﻌﺎﻣﺔ ﻓﻲ ﺗﺄﺳﻴﺲ ﺷﺮﻛﺔ ﺳﻮق
اﻟﻜﺮﺑﻮن اﻟﻄﻮﻋﻲ اﻻﻗﻠﻴﻤﻴﺔ .
• اﻛﺘﻤﺎل أول إدراج ﻣﺰدوج وﻣﺘﺰاﻣﻦ ﺑﻴﻦ ﺷﺮﻛﺔ ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ وﺳﻮق أﺑﻮﻇﺒﻲ ﻟﻸوراق اﻟﻤﺎﻟﻴﺔ .
• وﻗﻌﺖ ﺷﺮﻛﺔ "واﻣﺾ"، إﺣﺪى اﻟﺸﺮﻛﺎت اﻟﺘﺎﺑﻌﺔ ﻟﻤﺠﻤﻮﻋﺔ ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ، اﺗﻔﺎﻗﻴﺔ ﺑﻴﻊ وﺷﺮاء ﻣﻊ ﺷﺮﻛﺔ
"اﻟﻤﺒﺎدرات اﻟﺜﺎﻧﻴﺔ ﻟﻼﺳﺘﺜﻤﺎر" ﻟﻼﺳﺘﺤﻮاذ ﻋﻠﻰ 51% ﻣﻦ رأس اﻟﻤﺎل اﻟﻤﺼﺪر ﻟﺸﺮﻛﺔ ﺷﺒﻜﺔ ﻣﺒﺎﺷﺮ اﻟﻤﺎﻟﻴﺔ "داﻳﺮﻛﺖ
إف إن" .
• إﻃﻼق إﻃﺎر ﻋﻤﻞ ﺻﺎﻧﻊ اﻟﺴﻮق ﻷﺳﻮاق اﻷﺳﻬﻢ واﻟﻤﺸﺘﻘﺎت اﻟﻤﺎﻟﻴﺔ، ﻛﺠﺰء ﻣﻦ ﺟﻬﻮد ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ ﻟﻀﻤﺎن
ﺗﻮﻓﺮ اﻟﺴﻴﻮﻟﺔ وزﻳﺎدة ﻛﻔﺎءة ﺗﺤﺪﻳﺪ اﻷﺳﻌﺎر ﺗﻤﺎﺷﻴﺎ ﻣﻊ دﻋﻢ ﻧﻤﻮ اﻟﺴﻮق اﻟﻤﺎﻟﻴﺔ اﻟﺴﻌﻮدﻳﺔ ﺗﺤﻘﻴﻘﺎ ﻟﻤﺴﺘﻬﺪﻓﺎت
رؤﻳﺔ اﻟﻤﻤﻠﻜﺔ 2030.
اﻟﺮﻳﺎض، اﻟﻤﻤﻠﻜﺔ اﻟﻌﺮﺑﻴﺔ اﻟﺴﻌﻮدﻳﺔ، 26 ﻓﺒﺮاﻳﺮ 2023م: أﻋﻠﻨﺖ ﺷﺮﻛﺔ ﻣﺠﻤﻮﻋﺔ ﺗﺪاول اﻟﺴﻌﻮدﻳﺔ اﻟﻘﺎﺑﻀﺔ
)"اﻟﺸﺮﻛﺔ" أو "اﻟﻤﺠﻤﻮﻋﺔ"(، اﻟﺮاﺋﺪة ﻓﻲ ﻣﺠﺎل اﻷﺳﻮاق اﻟﻤﺎﻟﻴﺔ ﻓﻲ ﻣﻨﻄﻘﺔ اﻟﺸ